## Retail Transaction Data

### Objective:
#### Use $1000 left in marketing budget to convert some physical store customers to the online store.by mail some of these customers coupons for them to use online.

In [2]:
#Read Dataset and Explore it set parse_date with Trans_date to set the appropriate date type to the selected column
import pandas as pd

df = pd.read_csv("Retail_Data_Transactions.csv", parse_dates=["trans_date"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB
None


From info of data we release that ther's no missing values and the data type is right 

In [15]:
# Group data by customer ID
group_by_customer =df.groupby("customer_id")

In [16]:
# Find The Last transaction occured by each Customer
last_transaction = group_by_customer["trans_date"].max()
last_transaction 

customer_id
CS1112   2015-01-14
CS1113   2015-02-09
CS1114   2015-02-12
CS1115   2015-03-05
CS1116   2014-08-25
            ...    
CS8996   2014-12-09
CS8997   2014-06-28
CS8998   2014-12-22
CS8999   2014-07-02
CS9000   2015-02-28
Name: trans_date, Length: 6889, dtype: datetime64[ns]

In [17]:
# select 5 customer Randomly 
last_transaction.sample(5)

customer_id
CS4392   2015-03-09
CS5171   2015-03-15
CS8401   2015-02-22
CS4123   2015-03-11
CS5345   2015-01-25
Name: trans_date, dtype: datetime64[ns]

### Identify churned customer is one who hasn't purchased anything since October 16 (three months).

In [21]:
best_churn=pd.DataFrame(last_transaction)

Add churned Column that should have the value of 1 if the customer has churned and 0  otherwise.

In [22]:
#Create a datetime object representing October 16, 2019
import datetime
cutoff_day=datetime.datetime(2019, 10, 16)

In [24]:
# Function to identify Customers According to Date 
best_churn["churned"] = best_churn["trans_date"].apply(
    lambda date: 1 if date < cutoff_day else 0)

In [33]:
# Get the number of Purchases For Each Customer using Size Function which count the number of elements along a given axis
# and select five random
best_churn["nr_of_transactions"] = group_by_customer.size()
best_churn.sample(5)

,churned,nr_of_transactions,amount_spent
customer_id,,,
CS2083,1,26.0,1632.0
CS5691,1,18.0,1344.0
CS7648,1,15.0,763.0
CS1219,1,21.0,1447.0
CS3696,1,21.0,1387.0


In [31]:
# Get the total amount of spent For Each Customer and select five random
best_churn["amount_spent"]=group_by_customer.sum()
best_churn.sample(5)

,trans_date,churned,nr_of_transactions,amount_spent
customer_id,,,,
CS7125,2013-12-12,1,8.0,447.0
CS8738,2014-11-23,1,16.0,854.0
CS5372,2014-10-07,1,24.0,1721.0
CS5628,2015-02-08,1,16.0,1093.0
CS3390,2015-01-07,1,17.0,1208.0


In [32]:
# Drop Trans_date column now
best_churn.drop("trans_date", axis="columns", inplace=True)

### To Find The Best Customers 
#### - Find a Ranking Mechanism 
#### - Determine a Treshold 

In [35]:

best_churn["scaled_tran"]=(best_churn["nr_of_transactions"]-best_churn["nr_of_transactions"].min())/(best_churn["nr_of_transactions"].max()-best_churn["nr_of_transactions"].min())

In [36]:
best_churn["scaled_amount"] = (best_churn["amount_spent"] \
                               -best_churn["amount_spent"].min()) \
                               / (best_churn["amount_spent"].max() \
                               - best_churn["amount_spent"].min())

Normalization Columns
min-max feature scaling The goal of this technique is to compare different scales in a meaningful way.


using Weighted Sum Model to classify customers by number as score from the following Formula

### 1- Ranking Mechanism
 sorting the customers by their score.

In [38]:
best_churn["score"]=100*(0.5*best_churn["scaled_tran"]+0.5*best_churn["scaled_amount"])

In [39]:
best_churn.sort_values("score", inplace=True, ascending=False)

In [40]:
best_churn

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
CS4424,1,39.0,2933.0,1.000000,1.000000,100.000000
CS4320,1,38.0,2647.0,0.971429,0.897270,93.434934
CS3799,1,36.0,2513.0,0.914286,0.849138,88.171182
CS5109,1,35.0,2506.0,0.885714,0.846624,86.616892
CS3805,1,35.0,2453.0,0.885714,0.827586,85.665025
...,...,...,...,...,...,...
CS7224,1,4.0,191.0,0.000000,0.015086,0.754310
CS8504,1,4.0,190.0,0.000000,0.014727,0.736351
CS8559,1,4.0,157.0,0.000000,0.002874,0.143678


### 2-  Determine Treshold

From your experience, you know that a 30% discount is already very enticing;
 - value of the coupon :the mean of the transactions and compute 30% of that
 - nr_of_customers : Divide the budget by the valueof Cupons



In [43]:
coupon=df["tran_amount"].mean()*0.3
coupon

19.4975736

We will Round it to 20$ so  The Value of Coupon is 20$

In [45]:
nr_of_customers=1000/coupon
nr_of_customers

51.28843314123969

We Will Send The coupon for the top 50 Churned Customers

In [48]:
top_50_churned=best_churn.loc[best_churn["churned"]==1].head(50)
top_50_churned

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
CS4424,1,39.0,2933.0,1.000000,1.000000,100.000000
CS4320,1,38.0,2647.0,0.971429,0.897270,93.434934
CS3799,1,36.0,2513.0,0.914286,0.849138,88.171182
CS5109,1,35.0,2506.0,0.885714,0.846624,86.616892
CS3805,1,35.0,2453.0,0.885714,0.827586,85.665025
CS5752,1,33.0,2612.0,0.828571,0.884698,85.663485
CS4074,1,34.0,2462.0,0.857143,0.830819,84.398091
CS4660,1,33.0,2527.0,0.828571,0.854167,84.136905
CS1215,1,35.0,2362.0,0.885714,0.794899,84.030686


In [49]:
top_50_churned.to_csv("best_customers.txt")

Finally We Save data of 50 churn customer will recieve Cupons to send it to Manager as text File